In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import re
import json
import h2o

from datetime import datetime

from utils import get_data, Config, clean_url, make_clickable, add_columns

from tree import make_tree

In [3]:
# Load all data (except unknown/manual testing)
# Original Desktop Runs+Re-Runs were performed between 2024-02-05 and 2024-02-12
# Additional browsers + more than one response_id per parsing URL was run between 2024-02-16 and TBD
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Browser".name != 'Unknown' AND "Result".created_at < '2024-02-19';
"""
df = get_data(Config(), initial_data)
df = add_columns(df)

Connecting to the PostgreSQL database...
Connection successful


In [4]:
df["raw_header"] = df["raw_header"].apply(bytes).apply(lambda x: json.loads(x.decode("utf-8"))).astype(str)

In [5]:
responses = """
SELECT * from "Response";
"""
responses = get_data(Config(), responses)

Connecting to the PostgreSQL database...
Connection successful


# Overview

In [6]:
# In total 2 runs + couple of "repeat"-mode runs such that all browsers have results for each test
# Safari has more results as we had to abort and restart the main run a couple of times
# Both basic + parsing mode
df.groupby(["automation_mode", "browser_id", "name", "version", "os", "headless_mode"])["id"].count().sort_values(ascending=False).to_frame().reset_index()

,automation_mode,browser_id,name,version,os,headless_mode,id
0,selenium,42,chrome,121,Ubuntu 22.04,headless-new,1126766
1,selenium,45,brave,v1.62.156 (121.0.6167.139),Ubuntu 22.04,headless-new,1054336
2,selenium,43,firefox,122,Ubuntu 22.04,headless,1052084
3,selenium,44,edge,121,Ubuntu 22.04,headless-new,1049168
4,selenium,48,chrome,120,Ubuntu 22.04,headless-new,716930
5,selenium,47,chrome,122,Ubuntu 22.04,headless-new,714115
6,selenium,49,firefox,121,Ubuntu 22.04,headless,698631
7,selenium,46,safari,17.3,macOS 14.3,real,417257


# Error + Timeout Analysis
- Only timeout exist, no other systematic failures
- Systematic timeout: TAO in Safari -> fixed
- Couple of random timeouts in other tests

In [7]:
# Timeout (2), Fail (1), Not-run (3)
filtered_df = df.groupby(["browser"])["test_status"].filter(lambda x: x.nunique() >= 2)
display(df[["browser", "test_status"]].loc[df.index.isin(filtered_df.index)].groupby(["browser"])["test_status"].value_counts().to_frame())

count
browser                                            test_status         
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... 0            1054327
                                                   2                  9
chrome Ubuntu 22.04 120 selenium headless-new      0             716914
                                                   2                 16
chrome Ubuntu 22.04 121 selenium headless-new      0            1126758
                                                   2                  8
chrome Ubuntu 22.04 122 selenium headless-new      0             714105
                                                   2                 10
edge Ubuntu 22.04 121 selenium headless-new        0            1049079
                                                   2                 89
firefox Ubuntu 22.04 121 selenium headless         0             698593
                                                   2                 38
firefox Ubuntu 22.04 122 selenium headless         0            1052045
                                                   2                 39
safari macOS 14.3 17.3 selenium real               0             416992
                                                   2                265

In [8]:
# Teststatus == 2 (timeout)

# Rare timeouts in all browsers; Safari some more than others
display(df[df["test_status"] == 2]["browser"].value_counts().to_frame())

# Mostly 302 redirects for basic (oac_iframe, oac_window.open), referrer_iframe 
# perfAPI_img (only Safari, fixed!)
# Others quite rare
display(df.loc[(df["test_status"] == 2)].groupby(["resp_type", "status_code", "test_name"])["browser"].value_counts().to_frame().sort_values(by="count", ascending=False))

,count
browser,
safari macOS 14.3 17.3 selenium real,265
edge Ubuntu 22.04 121 selenium headless-new,89
firefox Ubuntu 22.04 122 selenium headless,39
firefox Ubuntu 22.04 121 selenium headless,38
chrome Ubuntu 22.04 120 selenium headless-new,16
chrome Ubuntu 22.04 122 selenium headless-new,10
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,9
chrome Ubuntu 22.04 121 selenium headless-new,8


count
resp_type status_code test_name             browser                                                  
basic     200         referrer_iframe       safari macOS 14.3 17.3 selenium real                  118
          302         referrer_iframe       safari macOS 14.3 17.3 selenium real                   74
parsing   200         perfAPI_img           safari macOS 14.3 17.3 selenium real                   32
basic     200         oac_iframe            safari macOS 14.3 17.3 selenium real                   24
          302         oac_iframe            edge Ubuntu 22.04 121 selenium headless-new            20
parsing   200         framing_iframe        firefox Ubuntu 22.04 121 selenium headless             18
basic     200         referrer_iframe       edge Ubuntu 22.04 121 selenium headless-new            16
          302         oac_window.open       edge Ubuntu 22.04 121 selenium headless-new            16
parsing   200         framing_iframe        edge Ubuntu 22.04 121 selenium headless-new            14
basic     200         referrer_iframe       firefox Ubuntu 22.04 122 selenium headless             13
parsing   200         framing_iframe        firefox Ubuntu 22.04 122 selenium headless             12
                      fullscreen_iframe     edge Ubuntu 22.04 121 selenium headless-new            12
basic     302         oac_iframe            safari macOS 14.3 17.3 selenium real                   10
          200         referrer_iframe       firefox Ubuntu 22.04 121 selenium headless             10
parsing   200         fullscreen_iframe     chrome Ubuntu 22.04 120 selenium headless-new          10
basic     302         referrer_iframe       edge Ubuntu 22.04 121 selenium headless-new             9
parsing   200         framing_iframe        brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      7
basic     302         oac_window.open       safari macOS 14.3 17.3 selenium real                    6
parsing   200         fullscreen_iframe     firefox Ubuntu 22.04 122 selenium headless              5
                                            chrome Ubuntu 22.04 122 selenium headless-new           5
                      framing_iframe        chrome Ubuntu 22.04 122 selenium headless-new           4
                      fullscreen_iframe     chrome Ubuntu 22.04 121 selenium headless-new           4
basic     200         referrer_iframe       chrome Ubuntu 22.04 120 selenium headless-new           4
          302         referrer_iframe       firefox Ubuntu 22.04 122 selenium headless              3
                      oac_window.open       firefox Ubuntu 22.04 122 selenium headless              3
                      referrer_iframe       firefox Ubuntu 22.04 121 selenium headless              3
parsing   200         fetch_GET             firefox Ubuntu 22.04 121 selenium headless              3
                      upgradeHSTS_subdomain firefox Ubuntu 22.04 121 selenium headless              2
                      framing_iframe        chrome Ubuntu 22.04 121 selenium headless-new           2
basic     302         oac_iframe            firefox Ubuntu 22.04 122 selenium headless              2
          200         referrer_iframe       brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                                            chrome Ubuntu 22.04 122 selenium headless-new           1
parsing   200         accesswindow_direct   chrome Ubuntu 22.04 121 selenium headless-new           1
                                            edge Ubuntu 22.04 121 selenium headless-new             1
basic     302         oac_window.open       firefox Ubuntu 22.04 121 selenium headless              1
parsing   200         accesswindow_direct   firefox Ubuntu 22.04 122 selenium headless              1
                      framing_iframe        safari macOS 14.3 17.3 selenium real                    1
                      imgloading_iframe     chrome Ubuntu 22.04 120 selenium headless-new           1
                

# General Test Statistics

In [9]:
df[["browser", "resp_type", "test_name", "relation_info", "response_id", "org_scheme", "org_host", "resp_scheme", "resp_host"]].drop_duplicates().groupby(["resp_type", "browser"]).count()

test_name  \
resp_type browser                                                         
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      10456   
          chrome Ubuntu 22.04 120 selenium headless-new           10456   
          chrome Ubuntu 22.04 121 selenium headless-new           10456   
          chrome Ubuntu 22.04 122 selenium headless-new           10456   
          edge Ubuntu 22.04 121 selenium headless-new             10456   
          firefox Ubuntu 22.04 121 selenium headless              10456   
          firefox Ubuntu 22.04 122 selenium headless              10456   
          safari macOS 14.3 17.3 selenium real                    10456   
parsing   brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     168774   
          chrome Ubuntu 22.04 120 selenium headless-new          168774   
          chrome Ubuntu 22.04 121 selenium headless-new          168774   
          chrome Ubuntu 22.04 122 selenium headless-new          168774   
          edge Ubuntu 22.04 121 selenium headless-new            168774   
          firefox Ubuntu 22.04 121 selenium headless             168774   
          firefox Ubuntu 22.04 122 selenium headless             168774   
          safari macOS 14.3 17.3 selenium real                   168774   

                                                              relation_info  \
resp_type browser                                                             
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...          10456   
          chrome Ubuntu 22.04 120 selenium headless-new               10456   
          chrome Ubuntu 22.04 121 selenium headless-new               10456   
          chrome Ubuntu 22.04 122 selenium headless-new               10456   
          edge Ubuntu 22.04 121 selenium headless-new                 10456   
          firefox Ubuntu 22.04 121 selenium headless                  10456   
          firefox Ubuntu 22.04 122 selenium headless                  10456   
          safari macOS 14.3 17.3 selenium real                        10456   
parsing   brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...         168774   
          chrome Ubuntu 22.04 120 selenium headless-new              168774   
          chrome Ubuntu 22.04 121 selenium headless-new              168774   
          chrome Ubuntu 22.04 122 selenium headless-new              168774   
          edge Ubuntu 22.04 121 selenium headless-new                168774   
          firefox Ubuntu 22.04 121 selenium headless                 168774   
          firefox Ubuntu 22.04 122 selenium headless                 168774   
          safari macOS 14.3 17.3 selenium real                       168774   

                                                              response_id  \
resp_type browser                                                           
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...        10456   
          chrome Ubuntu 22.04 120 selenium headless-new             10456   
          chrome Ubuntu 22.04 121 selenium headless-new             10456   
          chrome Ubuntu 22.04 122 selenium headless-new             10456   
          edge Ubuntu 22.04 121 selenium headless-new               10456   
          firefox Ubuntu 22.04 121 selenium headless                10456   
          firefox Ubuntu 22.04 122 selenium headless                10456   
          safari macOS 14.3 17.3 selenium real                      10456   
parsing   brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...       168774   
          chrome Ubuntu 22.04 120 selenium headless-new            168774   
          chrome Ubuntu 22.04 121 selenium headless-new            168774   
          chrome Ubuntu 22.04 122 selenium headless-new            168774   
          edge Ubuntu 22.04 121 selenium headless-new              168774   
          firefox Ubuntu 22.04 121 selenium headless               168774   
          firefox Ubuntu 22.04 122 selenium headless    

In [10]:
# Some browsers only used the more response_ids per page mode, thus these numbers don't say much
df.groupby("browser")["clean_url"].nunique()

browser
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    100726
chrome Ubuntu 22.04 120 selenium headless-new                           28713
chrome Ubuntu 22.04 121 selenium headless-new                          100726
chrome Ubuntu 22.04 122 selenium headless-new                           27889
edge Ubuntu 22.04 121 selenium headless-new                            100726
firefox Ubuntu 22.04 121 selenium headless                              29687
firefox Ubuntu 22.04 122 selenium headless                             100726
safari macOS 14.3 17.3 selenium real                                    85492
Name: clean_url, dtype: int64

In [11]:
df.groupby("browser")["response_id"].nunique()

browser
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    44415
chrome Ubuntu 22.04 120 selenium headless-new                          44415
chrome Ubuntu 22.04 121 selenium headless-new                          44415
chrome Ubuntu 22.04 122 selenium headless-new                          44415
edge Ubuntu 22.04 121 selenium headless-new                            44415
firefox Ubuntu 22.04 121 selenium headless                             44415
firefox Ubuntu 22.04 122 selenium headless                             44415
safari macOS 14.3 17.3 selenium real                                   44415
Name: response_id, dtype: int64

In [12]:
# Make sure that all responses actually have responses
# (create_repeat.py assumes that at least one browser collected the results)

r1 = df.groupby("label")["response_id"].nunique().sort_values()
r2 = responses.loc[responses["resp_type"] != "debug"].groupby("label")["id"].nunique().sort_values()

pd.concat([r1, r2], axis=1)

,response_id,id
label,,
CORS,6,6
CORS-ACAC,864,864
CSPvsXFO,1447,1447
CORS-ACAM,1576,1576
CORS-ACEH,1577,1577
CORS-ACAH,1577,1577
TAO,1614,1614
OAC,1713,1713
CORS-ACAO,2465,2465


In [13]:
df.groupby(["resp_type", "test_name", "relation_info"])["response_id"].nunique().to_frame()

response_id
resp_type test_name                     relation_info              
basic     accesswindow_direct           direct                   11
          fetch_GET                     credentials               6
                                        custom_headers            6
                                        simple                    6
          fetch_TEST                    custom_method             6
          framing_embed                 direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_iframe                direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_object                direct                   35
                                        nested                   35
                                        sandbox                  35
          fullscreen_iframe             child                    24
                                        child_allow              24
                                        child_sandbox            24
                                        direct                   24
          imgloading_iframe             direct                   17
          oac_iframe                    direct                   14
                                        sandbox                  14
          oac_window.open               window.open              14
          perfAPI_img                   direct                   13
          referrer_iframe               iframe                   16
                                        window.open              16
          script_execution_iframe       direct                   17
                                        sandbox                  17
          subresourceloadingCOEP_img    direct                   11
                                        sandbox                  11
          subresourceloadingCORP_img    direct                   11
                                        nested                   11
                                        sandbox                  11
          subresourceloadingCORP_object direct                   11
          upgradeHSTS_direct            direct                   14
          upgradeHSTS_subdomain         subdomain                14
parsing   accesswindow_direct           direct                 2656
          fetch_GET                     credentials            8059
                                        custom_headers         8059
                                        simple                 8059
          fetch_TEST                    custom_method          8059
          framing_iframe                direct                 8780
                                        nested                 8780
          fullscreen_iframe             child_allow            3880
                                        direct                 3880
          imgloading_iframe             direct                 2710
          oac_window.open               window.open            1699
          perfAPI_img                   direct                 1601
          referrer_iframe               iframe                 2797
          script_execution_iframe       direct                 2726
          subresourceloadingCOEP_img    direct                 2637
          subresourceloadingCORP_img    direct                 3357
          upgradeHSTS_direct            direct                 3324
          upgradeHSTS_subdomain         subdomain              3324

# Stability
- Basic tests:
    - 21/(10456x5) tests with different outcomes!
    - 13x firefox subresourceloadingCOEP_img (random behavior bug)
    - 6x timeout vs message 4(x edge, 2x brave)
    - 1x brave referrer different outcome
    - 1x safari COEP different outcome?
- Parsing tests:
    - 755/(168774x5) tests with different outcomes!
    - quite a lot! last time we only had 1 single test with a different outcome? (We used different versions of the browsers)
    - 3x accesswindow_direct (null vs accessible), 1x fullscreen_iframe (timeout vs message), 1x img_loading_iframe (timeout vs message)
    - All others (700+) upgradeHSTS_subdomain and upgradeHSTS_direct
        - Prevalence: `brave ~= edge > chrome >> firefox >> safari`
        - Hints at systematic issue: maybe the reset did not work or something similar?
        - Mostly timeout vs redirect=false; however some are also timeout vs redirect=false vs redirect=true!
        - Timeout maybe related to the HTTPS upgrade feature in chromium-based browsers which can take some time (>5s?)? (Non-Authoritative-Reason: HttpsUpgrades)
        - Unclear how redirect true vs false could happen? Maybe HSTS clearing did not work?
        - Manual testing always results in redirect=false for all tried clean_urls!

### Update: multiple response_ids per parsing URL test
- 2599+ test with different outcomes?!
- mostly timeouts?: Firefox fetch, other browsers subresourceloadingCOEP_img
- maybe we have too many tests per page/or the timeout is not high enough
- OT: if running `desktop_selenium.py` in the repeat mode, one has to increate `--timeout_task` (15*100 = 1500s > 1000) or decrease `--max_urls_until_restart` as we ran the repeat tests with a 3xTIMEOUT

In [14]:
# Remove all tests that timed out (test_status == 0), each test has at least one result that did not time out in each browser
df_org = df
df = df.loc[df["test_status"] == 0]

In [15]:
for group_name, resp_type_group in df.groupby("resp_type"):
    if group_name == "basic":
        continue
    print(group_name)
    stab = resp_type_group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])[["outcome_str"]].agg(["count", "nunique"])
    # Tests with more than one outcome
    diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
    print(f"{len(diff_outcomes)} tests have different outcomes!")
    display(diff_outcomes.reset_index()["browser"].value_counts().to_frame())
    display(diff_outcomes.reset_index()[["test_name", "browser"]].value_counts().to_frame())

    diff_with_responses = diff_outcomes.reset_index().groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "browser"])["response_id"].agg(list).to_frame().reset_index()
    
    for id, (test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, browser, response_ids) in diff_with_responses.iterrows():
        # TODO: shortcut, only show the first result for each test_name/browser; results are usually always the same!
        print(test_name, response_ids)
        rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"].isin(response_ids)) & (df["browser"] == browser)]
        with pd.option_context("display.max_colwidth", 200):
            display(rows[["outcome_str", "browser"]].value_counts().to_frame())
            disp = rows.drop_duplicates(subset="outcome_str")[["browser", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
            disp = disp.style.format({'full_url': make_clickable})
            display(disp)
        # input("Continue!")

parsing
5154 tests have different outcomes!


,count
browser,
firefox Ubuntu 22.04 121 selenium headless,1205
firefox Ubuntu 22.04 122 selenium headless,1001
chrome Ubuntu 22.04 122 selenium headless-new,935
edge Ubuntu 22.04 121 selenium headless-new,691
chrome Ubuntu 22.04 121 selenium headless-new,532
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,512
chrome Ubuntu 22.04 120 selenium headless-new,272
safari macOS 14.3 17.3 selenium real,6


count
(test_name, )              (browser, )                                              
fetch_GET                  firefox Ubuntu 22.04 121 selenium headless            977
                           firefox Ubuntu 22.04 122 selenium headless            754
subresourceloadingCOEP_img chrome Ubuntu 22.04 122 selenium headless-new         536
upgradeHSTS_subdomain      edge Ubuntu 22.04 121 selenium headless-new           287
subresourceloadingCOEP_img chrome Ubuntu 22.04 121 selenium headless-new         277
imgloading_iframe          chrome Ubuntu 22.04 122 selenium headless-new         277
upgradeHSTS_subdomain      brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...    272
subresourceloadingCOEP_img edge Ubuntu 22.04 121 selenium headless-new           238
upgradeHSTS_direct         brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...    232
subresourceloadingCOEP_img chrome Ubuntu 22.04 120 selenium headless-new         165
upgradeHSTS_direct         edge Ubuntu 22.04 121 selenium headless-new           143
fetch_TEST                 firefox Ubuntu 22.04 122 selenium headless            106
upgradeHSTS_subdomain      chrome Ubuntu 22.04 121 selenium headless-new          93
fetch_TEST                 firefox Ubuntu 22.04 121 selenium headless             92
upgradeHSTS_direct         chrome Ubuntu 22.04 121 selenium headless-new          86
imgloading_iframe          chrome Ubuntu 22.04 121 selenium headless-new          71
subresourceloadingCOEP_img firefox Ubuntu 22.04 122 selenium headless             67
imgloading_iframe          chrome Ubuntu 22.04 120 selenium headless-new          65
upgradeHSTS_subdomain      chrome Ubuntu 22.04 122 selenium headless-new          63
subresourceloadingCOEP_img firefox Ubuntu 22.04 121 selenium headless             60
upgradeHSTS_direct         firefox Ubuntu 22.04 122 selenium headless             57
                           firefox Ubuntu 22.04 121 selenium headless             50
                           chrome Ubuntu 22.04 122 selenium headless-new          35
upgradeHSTS_subdomain      chrome Ubuntu 22.04 120 selenium headless-new          27
                           firefox Ubuntu 22.04 121 selenium headless             26
framing_iframe             chrome Ubuntu 22.04 122 selenium headless-new          22
imgloading_iframe          edge Ubuntu 22.04 121 selenium headless-new            18
upgradeHSTS_subdomain      firefox Ubuntu 22.04 122 selenium headless             17
upgradeHSTS_direct         chrome Ubuntu 22.04 120 selenium headless-new          13
framing_iframe             brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      4
upgradeHSTS_direct         safari macOS 14.3 17.3 selenium real                    3
upgradeHSTS_subdomain      safari macOS 14.3 17.3 selenium real                    2
fullscreen_iframe          chrome Ubuntu 22.04 121 selenium headless-new           2
accesswindow_direct        edge Ubuntu 22.04 121 selenium headless-new             2
referrer_iframe            edge Ubuntu 22.04 121 selenium headless-new             2
fullscreen_iframe          brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      2
                           edge Ubuntu 22.04 121 selenium headless-new             1
framing_iframe             chrome Ubuntu 22.04 120 selenium headless-new           1
accesswindow_direct        chrome Ubuntu 22.04 121 selenium headless-new           1
fullscreen_iframe          chrome Ubuntu 22.04 122 selenium headless-new           1
script_execution_iframe    chrome Ubuntu 22.04 120 selenium headless-new           1
referrer_iframe            chrome Ubuntu 22.04 121 selenium headless-new           1
script_execution_iframe    brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
perfAPI_img                chrome Ubuntu 22.04 121 selenium headless-new           1
referrer_iframe            brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                           chrome Ubuntu 22.04 122 selenium headless-new           1

accesswindow_direct [22059, 22478]


,,count
outcome_str,browser,
{'window.open.opener': 'null'},edge Ubuntu 22.04 121 selenium headless-new,10
"{'window.open.opener': 'object ""[object Window]""'}",edge Ubuntu 22.04 121 selenium headless-new,2


...

,,count
outcome_str,browser,
{'response.redirected': False},firefox Ubuntu 22.04 122 selenium headless,85
Fetch timed out,firefox Ubuntu 22.04 122 selenium headless,17


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
643095,firefox Ubuntu 22.04 122 selenium headless,{'response.redirected': False},"[['strict-transport-security', 'max-a?ge=60']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=32149&last_id=32149&scheme=http
683142,firefox Ubuntu 22.04 122 selenium headless,Fetch timed out,"[['strict-transport-security', 'max-a?ge=60']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=32149&last_id=32149&scheme=http


upgradeHSTS_subdomain [29963, 33015]


,,count
outcome_str,browser,
Fetch timed out,safari macOS 14.3 17.3 selenium real,2
{'response.redirected': False},safari macOS 14.3 17.3 selenium real,2


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
714202,safari macOS 14.3 17.3 selenium real,{'response.redirected': False},"[['strict-transport-securityO', 'max-age=0']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=29963&last_id=29963&scheme=http
723487,safari macOS 14.3 17.3 selenium real,Fetch timed out,"[['strict-transport-securityO', 'max-age=0']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=29963&last_id=29963&scheme=http


## More stability?!
- How many outcomes did we collect per test in the different browsers? Also shows tests with only one outcome
- How many of them are different?

In [16]:
final_df = pd.DataFrame()

for group, d in df.groupby("browser"):
    # Calculate counts and unique counts
    aggs = ["count"] # ["count", "nunique"]
    res = d.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])["outcome_str"].agg(aggs)
    # Get value counts for each combination of counts and unique counts
    counts = res.value_counts().to_frame()
    counts = counts.rename(columns={"count": group})

    only_once = res.loc[res["count"] == 1]
    if len(only_once) < 200:
        print(group)
        display(only_once.reset_index()[["test_name", "relation_info"]].value_counts().to_frame())

    final_df = pd.concat([final_df, counts], axis=1)

display(final_df)

brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new


,,count
test_name,relation_info,


chrome Ubuntu 22.04 120 selenium headless-new


,,count
test_name,relation_info,


chrome Ubuntu 22.04 121 selenium headless-new


...

firefox Ubuntu 22.04 122 selenium headless


,,count
test_name,relation_info,


safari macOS 14.3 17.3 selenium real


,,count
test_name,relation_info,
accesswindow_direct,direct,13
referrer_iframe,iframe,9
perfAPI_img,direct,8
referrer_iframe,window.open,6


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,safari macOS 14.3 17.3 selenium real
count,,,,,,,,
6,168636.0,NaN,96480.0,NaN,163446.0,NaN,166372.0,NaN
4,10457.0,179214.0,10456.0,176415.0,10404.0,160882.0,10437.0,10779.0
5,136.0,5.0,2.0,NaN,5328.0,11.0,2402.0,NaN
3,1.0,11.0,NaN,2815.0,43.0,18336.0,17.0,37016.0
7,NaN,NaN,72292.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,9.0,1.0,2.0,131396.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0


In [17]:
grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].count().reset_index()

with pd.option_context("display.max_rows", 202):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["outcome_str"].agg(["mean", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0])

,mean,count,sum
count,256.000000,256.000000,256.000000
mean,4.343569,5600.925781,26675.050781
std,1.023030,10301.094819,52435.087318
min,1.997502,28.000000,112.000000
25%,4.000000,284.000000,1136.000000
50%,4.000000,1680.000000,6720.000000
75%,4.000000,5487.500000,27902.000000
max,6.500000,48354.000000,314301.000000


mean  \
browser                                       resp_type test_name                           
chrome Ubuntu 22.04 121 selenium headless-new parsing   fetch_GET                6.500000   
                                                        fetch_TEST               6.500000   
                                                        perfAPI_img              6.500000   
                                                        referrer_iframe          6.500000   
                                                        script_execution_iframe  6.500000   
...                                                                                   ...   
safari macOS 14.3 17.3 selenium real          parsing   script_execution_iframe  2.000000   
                                                        imgloading_iframe        2.000000   
                                                        fullscreen_iframe        2.000000   
                                                        accesswindow_direct      1.997553   
                                                        perfAPI_img              1.997502   

                                                                                 count  \
browser                                       resp_type test_name                        
chrome Ubuntu 22.04 121 selenium headless-new parsing   fetch_GET                48354   
                                                        fetch_TEST               16118   
                                                        perfAPI_img               3202   
                                                        referrer_iframe           5594   
                                                        script_execution_iframe   5452   
...                                                                                ...   
safari macOS 14.3 17.3 selenium real          parsing   script_execution_iframe   5452   
                                                        imgloading_iframe         5420   
                                                        fullscreen_iframe        15520   
                                                        accesswindow_direct       5312   
                                                        perfAPI_img               3202   

                                                                                    sum  
browser                                       resp_type test_name                        
chrome Ubuntu 22.04 121 selenium headless-new parsing   fetch_GET                314301  
                                                        fetch_TEST               104767  
                                                        perfAPI_img               20813  
                                                        referrer_iframe           36361  
                                                        script_execution_iframe   35438  
...                                                                                 ...  
safari macOS 14.3 17.3 selenium real          parsing   script_execution_iframe   10904  
                                                        imgloading_iframe         10840  
                                                        fullscreen_iframe         31040  
                                                        accesswindow_direct       10611  
                                                        perfAPI_img                6396  

[256 rows x 3 columns]

In [18]:
# Different outcomes (in percentage for a test group (test_name)
grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].nunique().reset_index()

grouped["diff_outcome"] = grouped["outcome_str"] > 1
with pd.option_context("display.max_rows", 50):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["diff_outcome"].agg(["mean", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0])

,mean,count,sum
count,256.000000,256.000000,256.000000
mean,0.003251,5600.925781,20.410156
std,0.012428,10301.094819,93.730191
min,0.000000,28.000000,0.000000
25%,0.000000,284.000000,0.000000
50%,0.000000,1680.000000,0.000000
75%,0.000000,5487.500000,0.000000
max,0.101631,48354.000000,977.000000


mean  \
browser                                            resp_type test_name                              
chrome Ubuntu 22.04 122 selenium headless-new      parsing   subresourceloadingCOEP_img  0.101631   
firefox Ubuntu 22.04 122 selenium headless         basic     subresourceloadingCOEP_img  0.093750   
firefox Ubuntu 22.04 121 selenium headless         basic     subresourceloadingCOEP_img  0.076705   
chrome Ubuntu 22.04 121 selenium headless-new      parsing   subresourceloadingCOEP_img  0.052522   
chrome Ubuntu 22.04 122 selenium headless-new      parsing   imgloading_iframe           0.051107   
...                                                                                           ...   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... parsing   fullscreen_iframe           0.000129   
                                                             framing_iframe              0.000114   
edge Ubuntu 22.04 121 selenium headless-new        parsing   fullscreen_iframe           0.000064   
chrome Ubuntu 22.04 122 selenium headless-new      parsing   fullscreen_iframe           0.000064   
chrome Ubuntu 22.04 120 selenium headless-new      parsing   framing_iframe              0.000028   

                                                                                         count  \
browser                                            resp_type test_name                           
chrome Ubuntu 22.04 122 selenium headless-new      parsing   subresourceloadingCOEP_img   5274   
firefox Ubuntu 22.04 122 selenium headless         basic     subresourceloadingCOEP_img    352   
firefox Ubuntu 22.04 121 selenium headless         basic     subresourceloadingCOEP_img    352   
chrome Ubuntu 22.04 121 selenium headless-new      parsing   subresourceloadingCOEP_img   5274   
chrome Ubuntu 22.04 122 selenium headless-new      parsing   imgloading_iframe            5420   
...                                                                                        ...   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... parsing   fullscreen_iframe           15520   
                                                             framing_iframe              35120   
edge Ubuntu 22.04 121 selenium headless-new        parsing   fullscreen_iframe           15520   
chrome Ubuntu 22.04 122 selenium headless-new      parsing   fullscreen_iframe           15520   
chrome Ubuntu 22.04 120 selenium headless-new      parsing   framing_iframe              35120   

                                                                                         sum  
browser                                            resp_type test_name                        
chrome Ubuntu 22.04 122 selenium headless-new      parsing   subresourceloadingCOEP_img  536  
firefox Ubuntu 22.04 122 selenium headless         basic     subresourceloadingCOEP_img   33  
firefox Ubuntu 22.04 121 selenium headless         basic     subresourceloadingCOEP_img   27  
chrome Ubuntu 22.04 121 selenium headless-new      parsing   subresourceloadingCOEP_img  277  
chrome Ubuntu 22.04 122 selenium headless-new      parsing   imgloading_iframe           277  
...                                                                                      ...  
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... parsing   fullscreen_iframe             2  
                                                             framing_iframe                4  
edge Ubuntu 22.04 121 selenium headless-new        parsing   fullscreen_iframe             1  
chrome Ubuntu 22.04 122 selenium headless-new      parsing   fullscreen_iframe             1  
chrome Ubuntu 22.04 120 selenium headless-new      parsing   framing_iframe                1  

[56 rows x 3 columns]

In [19]:
# Run the test groups with many different outcomes again?
# Groups with non-negligble different outcomes are subresourceloadingCOEP_img, upgradeHSTS_subdomain, upgradeHSTS_direct, imgloading_iframe, fetch_TEST, fetch_GET
test_files_to_repeat =  ["fetch-cors.sub.html", "subresource-loading-coep.sub.html", "subresource-loading-csp.sub.html", "upgrade-hsts.sub.html"]

# TODO: implement utils.py/desktop_selenium mode to only rerun these files?!
# For now, simply run everything again to get some more results for stability?!

In [20]:
# Majority voting for the tests with multiple outcomes
df.sample(10000).groupby(["test_name", "relation_info", "browser", "org_host", "resp_scheme", "org_origin", "resp_origin", "response_id"])["outcome_str"].agg([pd.Series.mode, "nunique", "count"]).reset_index().sort_values("count")

,test_name,relation_info,browser,org_host,resp_scheme,org_origin,resp_origin,response_id,mode,nunique,count
6646,fullscreen_iframe,direct,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) ...,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://headers.webappsec.eu,36881,fullscreenEnabled: true,1,1
6637,fullscreen_iframe,direct,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) ...,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://headers.webappsec.eu,35660,fullscreenEnabled: true,1,1
6638,fullscreen_iframe,direct,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) ...,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://headers.webappsec.eu,36013,fullscreenEnabled: true,1,1
6639,fullscreen_iframe,direct,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) ...,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://headers.webappsec.eu,36058,fullscreenEnabled: true,1,1
6640,fullscreen_iframe,direct,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) ...,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://headers.webappsec.eu,36168,fullscreenEnabled: true,1,1
...,...,...,...,...,...,...,...,...,...,...,...
4629,framing_iframe,direct,edge Ubuntu 22.04 121 selenium headless-new,sub.headers.websec.saarland,https,http://sub.headers.websec.saarland,https://headers.webappsec.eu,4508,message send,1,2
6813,fullscreen_iframe,direct,chrome Ubuntu 22.04 122 selenium headless-new,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://headers.webappsec.eu,35452,fullscreenEnabled: true,1,2
813,fetch_GET,credentials,edge Ubuntu 22.04 121 selenium headless-new,sub.headers.websec.saarland,https,http://sub.headers.websec.saarland,https://headers.webappsec.eu,9495,"{'error': 'object ""TypeError: Failed to fetch""...",1,2
1693,fetch_GET,custom_headers,chrome Ubuntu 22.04 122 selenium headless-new,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://headers.webappsec.eu,12329,"{'error': 'null', 'headers': 'content-length,t...",1,2


# Browser differences!

In [21]:
df = df_org
df = df.loc[df["test_status"] == 0]

In [22]:
df_stab = df

# Only keep one row for each test
# If the test had non-deterministic results, use majority voting! (only works reliably if there a enough repetitions of the tests runs)

print("Original data entries", len(df))
#df = df.drop_duplicates(subset=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "outcome_str"])
#print("Only one row for deterministic tests", len(df))

# Majority voting (pd.Series.mode returns the most frequest item) (quite slow)
#df_mult_outcomes = df[df.duplicated(subset=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"], keep=False)]
df = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])["outcome_str"].agg(pd.Series.mode).apply(lambda x: x[0]).reset_index()
#df = pd.concat([df[~df.duplicated(subset=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"], keep=False)], df_mult_outcomes])
print("Only one row for each test (per browser; majority voting)", len(df))

Original data entries 6828813
Only one row for each test (per browser; majority voting) 1433837


In [23]:
# Most tests have exactly one outcome, quite some have two, couple have three!
res = df.groupby(["resp_type", "test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
res = res["outcome_str"].agg("nunique")
display(res.value_counts().to_frame())

KeyError: 'resp_type'

In [ ]:
# Basic:
# subresourceloadingCORP_object has different results for 100% of tests
# couple of others have 20%+ differences
# coupel has 0%

# Parsing:
# oac_window.open has 99%
# Other test groups have between 0.5 - 7.5% of tests with different outcomes
r = res.reset_index()
r["diff"] = r["outcome_str"] != 1
r.groupby(["resp_type", "test_name", "relation_info"])["diff"].agg(["count", "sum", "mean"]).sort_values("mean", ascending=False)

In [ ]:
# Select only test rows with more than one outcome
tests_with_mult_outcomes = r.loc[r["diff"]].set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
df_mult = df.loc[df[["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]].apply(tuple, axis=1).isin(tests_with_mult_outcomes.index)]

print(f"All test rows: {len(df)}, Rows with more than one outcome: {len(df_mult)}")

In [ ]:
# Display difference groups
for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    df_list = []
    for _, one_test in group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]):
        browsers = one_test.groupby("outcome_str")["browser"].unique().apply(list).apply(sorted).to_frame()
        new = True
        for df_b in df_list:
            if df_b.equals(browsers):
                new = False
        if new:
            df_list.append(browsers)
    print(len(df_list))
    for df_b in df_list:
        with pd.option_context("display.max_colwidth", 1000):
            display(df_b)
    #input("Continue!")

In [ ]:
# Other approach to list the difference groups
def get_uniques(df):
    browsers = f"{df['name'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    
    unique_outcomes = {}
    group.groupby(["name"])["outcome_str"].value_counts(normalize=False).reset_index().groupby(["outcome_str", "count"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        display(d)
        print()

In [ ]:
# Function to calculate absolute and percentage overlap
def calculate_overlap(df, col1, col2):
    df = df[[col1, col2]].dropna()
    df.columns.values[0] = 0
    df.columns.values[1] = 1
    absolute_overlap = (df[0] == df[1]).sum()
    unique_rows = len(df)
    diff_rows = unique_rows - absolute_overlap
    percentage_diff = (diff_rows / unique_rows) * 100
    return diff_rows, percentage_diff, unique_rows


def display_overlap(df, show_all=True, name="All"):
    print(name)
    sim_frame = df.set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])["outcome_str"].unstack().astype("str")
    sim_frame = sim_frame.reset_index().drop(columns=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
    # Create a dictionary to store results
    results = {}
    matrix = {}
    
    # Iterate through all pairs of columns
    for i, col1 in enumerate(sim_frame.columns):
        for col2 in sim_frame.columns:
            key = f"{col1}_vs_{col2}"
            results[key] = calculate_overlap(sim_frame, col1, col2)
            try:
                l = matrix[col1]
            except KeyError:
                l = {}
            # 0 for absolute count of differences, 1 for percentage
            l[col2] = results[key][0]
            matrix[col1] = l
            
    
    # Convert the results to a DataFrame for better visualization
    results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Absolute Diff', 'Percentage Diff', "Count"])
    
    # Display the results
    if show_all:
        display(results_df)
    
    # Display difference matrix
    with pd.option_context("display.max_columns", 28):
        if show_all:
            display(pd.DataFrame(matrix))
        else:
            display(pd.DataFrame(matrix).drop_duplicates().T)


display_overlap(df)

for name, group in df.groupby("test_name"):
    display_overlap(group, show_all=False, name=name)

In [ ]:
# How many rows exist for tree creation
df[["test_name", "label", "relation_info"]].value_counts()

In [ ]:
h2o.init(nthreads=50, max_mem_size="100G", log_level="WARN")
h2o.no_progress()


# Limit on responses that have more than one outcome!
# This will remove all the branches that are the same in all browsers (and make trees large and confusing)
condition = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])["outcome_str"].transform("nunique") != 1
tree_df = df.loc[condition]
print("Remove tests that are the same in all browsers", len(tree_df))
base_dir = f"trees/{datetime.today().strftime('%Y-%m-%dT%H:%M')}" 

for group_name, group in tree_df.groupby(["test_name", "relation_info"]):
    print(group_name)
    pred_cols = ["browser", "org_origin", "resp_origin", "status_code", "raw_header"]
    group = group[["outcome_str", *pred_cols]]
    tree = make_tree(group, pred_cols, group_name, base_dir)
    # print(tree)


# Manual analysis

In [ ]:
# Chromium-based treats host-source as case sensitive even though it should not? (URLs and CSP is case-insensitive!)
# Play around: https://observer.sectec.rocks/opg/iframe/?url=https://echo.sectec.rocks/echo/?content-security-policy=img-src%20HTTPS://echo.sectec.rocks&ecocnt_css=%3Cimg%20src=https://echo.sectec.rocks/%3E&content-type=text/html
# Issue is in the scheme match which should be case-insensitive but is in Chromium-based browsers https://w3c.github.io/webappsec-csp/#match-schemes
# host-part matching was fixed here https://issues.chromium.org/issues/41412675, seems like they forgot fixing scheme-sensitivity 
df.loc[df["raw_header"].str.contains("img-src HTTPS://")][["clean_url", "raw_header", "outcome_str", "browser"]].value_counts().to_frame()